In [6]:
from astropy.stats import sigma_clipped_stats
from photutils.datasets import load_star_image
from photutils.detection import DAOStarFinder

from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image

# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

In [7]:
matplotlib.rcParams['figure.dpi'] = 200
matplotlib.rcParams['figure.figsize'] = (12, 12)

In [11]:
cc_com_reduced = './data/cc_com/reduced/'
tz_boo_reduced = './data/tz_boo/reduced/'

def get_fluxes(images_dir, image_dim=(3056, 3056)):
    images = ImageFileCollection(images_dir)
    images = images.files_filtered(include_path=True)
    
    mask = np.ones(image_dim, dtype=bool)
    mask[1400:1600, 1400:1600] = False
    
    fluxes = []
    for i, path in np.ndenumerate(images):
        # print(path)
        image = CCDData.read(path)
        mean, media, stdev = sigma_clipped_stats(image.data, sigma=3.0)
        daofind = DAOStarFinder(fwhm=3.0, threshold=5.0*stdev)
        sources = daofind(image.data, mask=mask)
        image.header['FLUX'] = sources[0]['flux']
        image.write(path, overwrite=True)
        fluxes.append(sources[0]['flux'])
        
    return np.array(fluxes)
    

In [12]:
tzboo_fluxes = get_fluxes(tz_boo_reduced)
tzboo_fluxes

array([ 45.28386457,  70.99202022, 128.77656612, 102.93023116])

In [13]:
cccom_fluxes = get_fluxes(cc_com_reduced)

In [14]:
cccom_fluxes

array([3.31905206, 4.40771791, 5.946138  ])

```python
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture

positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.0)
norm = ImageNormalize(stretch=SqrtStretch())
# plt.imshow(tzboo3.data, cmap='Greys', origin='lower', norm=norm,
#            interpolation='nearest');

fig, ax = plt.subplots(figsize=(12, 12))

show_image(tzboo3.data, fig=fig, ax=ax)
apertures.plot(color='red', lw=10, alpha=0.5);
```